In [1]:
import os
import io
import cv2
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
from ultralytics import YOLO
from pytorch_tabnet.tab_model import TabNetClassifier
import torch


In [2]:

# === Model Setup ===
yolo_disease = YOLO("outputs/crop_diseases_aug_model/weights/best.pt", task="segment")
yolo_insect = YOLO("outputs/crop_insects_aug_model/weights/best.pt", task="detect")

tabnet_disease = TabNetClassifier()
tabnet_disease.load_model("disease_model_v1.zip")

tabnet_insect = TabNetClassifier()
tabnet_insect.load_model("insect_model_v1.zip")

try:
    # Load the same CSV your model was trained on
    df = pd.read_csv('synthetic_disease_presence_30_questions.csv')
    # Get the feature names by dropping the target column ('Disease_Present')
    disease_questions = df.drop('Disease_Present', axis=1).columns.tolist()
    print(f"✅ Successfully loaded {len(disease_questions)} questions from the dataset.")
except FileNotFoundError:
    print("❌ Error: 'synthetic_disease_presence_30_questions.csv' not found.")
    print("Please make sure the CSV file is in the correct directory.")
    exit()
except Exception as e:
    print(f"❌ An error occurred while loading questions: {e}")
    exit()

try:
    # Load the same CSV your model was trained on
    df = pd.read_csv('synthetic_insect_presence_30_questions.csv')
    # Get the feature names by dropping the target column ('Disease_Present')
    insect_questions = df.drop('Insect_Present', axis=1).columns.tolist()
    print(f"✅ Successfully loaded {len(insect_questions)} questions from the dataset.")
except FileNotFoundError:
    print("❌ Error: 'synthetic_insect_presence_30_questions.csv' not found.")
    print("Please make sure the CSV file is in the correct directory.")
    exit()
except Exception as e:
    print(f"❌ An error occurred while loading questions: {e}")
    exit()



✅ Successfully loaded 30 questions from the dataset.
✅ Successfully loaded 30 questions from the dataset.


c:\Users\moham\Agrithon\venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [3]:

# === Upload Widgets ===
disease_upload = widgets.FileUpload(accept='image/*', multiple=False, description="📤 Upload Crop Disease Image")
insect_upload = widgets.FileUpload(accept='image/*', multiple=False, description="📤 Upload Insect Image")


In [4]:
# === Toggle Questions ===
def create_questionnaire(questions):
    toggles = []
    for q in questions:
        toggles.append(widgets.ToggleButtons(
            options=["Yes", "No"],
            description=q,
            style={"description_width": "initial"},
            layout=widgets.Layout(width='100%')
        ))
    return toggles

disease_toggles = create_questionnaire(disease_questions)
insect_toggles = create_questionnaire(insect_questions)

run_btn = widgets.Button(description="🚀 Run Multimodal Prediction", button_style="success")
output = widgets.Output()


In [ ]:

# === Logic ===
def predict_yolo(model, image_np, task, conf_threshold=0.4):
    temp_path = "temp_input.jpg"
    cv2.imwrite(temp_path, image_np)
    results = model.predict(temp_path, save=False, conf=conf_threshold)
    
    if task == "segment":
        if results[0].masks is not None:
            return len(results[0].masks.data)
        else:
            return 0
    else:  # task == "detect"
        if results[0].boxes is not None:
            # Filter by confidence
            confs = results[0].boxes.conf.cpu().numpy()
            return np.sum(confs > conf_threshold)
        else:
            return 0

def predict_tabnet(model, toggles):
    # Convert answers into features
    features = [[1 if toggle.value == "Yes" else 0 for toggle in toggles]]
    features = np.array(features).astype(np.float32)
    
    # Convert to tensor
    features_tensor = torch.tensor(features).to(model.device)
    
    # Run prediction
    with torch.no_grad():
        preds = model.predict(features_tensor.cpu().numpy())  # TabNet uses numpy input
    return int(preds[0])

def on_run_clicked(b):
    with output:
        clear_output()
        if len(disease_upload.value) == 0 or len(insect_upload.value) == 0:
            print("⚠️ Please upload both disease and insect images.")
            return

        disease_file = disease_upload.value[0]
        insect_file = insect_upload.value[0]

        disease_img_bytes = disease_file['content']
        insect_img_bytes = insect_file['content']

        disease_image = np.array(Image.open(io.BytesIO(disease_img_bytes)).convert("RGB"))
        insect_image = np.array(Image.open(io.BytesIO(insect_img_bytes)).convert("RGB"))

        # === YOLO Prediction with Visualization
        disease_temp = "temp_disease.jpg"
        insect_temp = "temp_insect.jpg"
        cv2.imwrite(disease_temp, disease_image)
        cv2.imwrite(insect_temp, insect_image)

        # Run YOLO and get annotated image
        disease_results = yolo_disease.predict(disease_temp, save=False, conf=0.4)
        insect_results = yolo_insect.predict(insect_temp, save=False, conf=0.6)

        yolo_disease_output = len(disease_results[0].masks.data) if disease_results[0].masks else 0
        yolo_insect_output = len(insect_results[0].boxes.data) if insect_results[0].boxes else 0

        # Show YOLO visual result images
        print("🖼️ YOLO Prediction - Crop Disease")
        disease_annotated = disease_results[0].plot()  # BGR image with overlays
        display(Image.fromarray(cv2.cvtColor(disease_annotated, cv2.COLOR_BGR2RGB)))

        print("🖼️ YOLO Prediction - Insect Detection")
        insect_annotated = insect_results[0].plot()
        display(Image.fromarray(cv2.cvtColor(insect_annotated, cv2.COLOR_BGR2RGB)))

        # === TabNet Prediction
        tabnet_disease_output = predict_tabnet(tabnet_disease, disease_toggles)
        tabnet_insect_output = predict_tabnet(tabnet_insect, insect_toggles)

        # === Fusion Logic
        YOLO_BOX_THRESH = 1
        TABNET_CONF_THRESH = 1

        disease_present = (yolo_disease_output >= YOLO_BOX_THRESH) and (tabnet_disease_output >= TABNET_CONF_THRESH)
        insect_present = (yolo_insect_output >= YOLO_BOX_THRESH) and (tabnet_insect_output >= TABNET_CONF_THRESH)

        # === Final Display ===
        print("\n🔍 Multimodal Fusion Results")
        print("🦠 Disease:")
        print(f"🔬 YOLO Masks Detected: {yolo_disease_output}")
        print(f"🧠 TabNet Prediction: {'Present' if tabnet_disease_output == 1 else 'Not Present'}")
        print(f"✅ Final: {'Disease Present' if disease_present else 'No Disease Detected'}")

        print("\n🐛 Insect:")
        print(f"🔬 YOLO Boxes Detected: {yolo_insect_output}")
        print(f"🧠 TabNet Prediction: {'Present' if tabnet_insect_output == 1 else 'Not Present'}")
        print(f"✅ Final: {'Insect Present' if insect_present else 'No Insect Detected'}")


run_btn.on_click(on_run_clicked)



In [6]:
# === Display All ===
display(widgets.HTML("<h3>Upload images and answer the questions below:</h3>"))
display(widgets.HBox([disease_upload, insect_upload]))
display(widgets.HTML("<h4>🦠 Crop Disease Questions</h4>"))
display(widgets.VBox(disease_toggles))
display(widgets.HTML("<h4>🐛 Crop Insect Questions</h4>"))
display(widgets.VBox(insect_toggles))
display(run_btn)
display(output)

HTML(value='<h3>Upload images and answer the questions below:</h3>')

HTML(value='<h4>🦠 Crop Disease Questions</h4>')

HTML(value='<h4>🐛 Crop Insect Questions</h4>')

Button(button_style='success', description='🚀 Run Multimodal Prediction', style=ButtonStyle())

Output()

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from ultralytics import YOLO
from pytorch_tabnet.tab_model import TabNetClassifier
import os

# === Step 1: Load CSVs ===
try:
    disease_df = pd.read_csv("synthetic_disease_presence_30_questions.csv")
    insect_df = pd.read_csv("synthetic_insect_presence_30_questions.csv")
    print("✅ CSV datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Error loading CSV: {e}")
    print("Please make sure the synthetic dataset files are in the correct directory.")
    exit()

# Simulate the 'Image' column, as it's missing from the synthetic data
disease_df['Image'] = [f'disease_img_{i}.jpg' for i in range(len(disease_df))]
insect_df['Image'] = [f'insect_img_{i}.jpg' for i in range(len(insect_df))]


# === Step 2: Encode Data Correctly ===
disease_df_encoded = disease_df.copy()
insect_df_encoded = insect_df.copy()

label_col_disease = 'Disease_Present'
label_col_insect = 'Insect_Present'

# Encode all feature columns (questions)
question_cols_disease = disease_df.columns[:-2]
question_cols_insect = insect_df.columns[:-2]

for col in question_cols_disease:
    disease_df_encoded[col] = disease_df_encoded[col].map({'Yes': 1, 'No': 0})

for col in question_cols_insect:
    insect_df_encoded[col] = insect_df_encoded[col].map({'Yes': 1, 'No': 0})

# Use the correct mapping for labels
disease_df_encoded[label_col_disease] = disease_df_encoded[label_col_disease].map({'Present': 1, 'Not Present': 0})
insect_df_encoded[label_col_insect] = insect_df_encoded[label_col_insect].map({'Present': 1, 'Not Present': 0})

# Drop rows where mapping might have failed
disease_df_encoded.dropna(inplace=True)
insect_df_encoded.dropna(inplace=True)

# Convert label to int
disease_df_encoded[label_col_disease] = disease_df_encoded[label_col_disease].astype(int)
insect_df_encoded[label_col_insect] = insect_df_encoded[label_col_insect].astype(int)

# === Step 3: Stratified Split ===
_, val_disease = train_test_split(disease_df_encoded, test_size=0.2,
                                  stratify=disease_df_encoded[label_col_disease], random_state=42)

_, val_insect = train_test_split(insect_df_encoded, test_size=0.2,
                                  stratify=insect_df_encoded[label_col_insect], random_state=42)

# === Step 4: TabNet Prediction ===
X_val_disease = val_disease[question_cols_disease].values.astype(np.float32)
y_val_disease = val_disease[label_col_disease].values

X_val_insect = val_insect[question_cols_insect].values.astype(np.float32)
y_val_insect = val_insect[label_col_insect].values

# Load TabNet Models
try:
    clf_disease = TabNetClassifier()
    clf_disease.load_model("disease_model_v1.zip")
    clf_insect = TabNetClassifier()
    clf_insect.load_model("insect_model_v1.zip")
except Exception as e:
    print(f"❌ Error loading TabNet model: {e}")
    exit()

tabnet_disease_preds = clf_disease.predict(X_val_disease)
tabnet_insect_preds = clf_insect.predict(X_val_insect)

# === Step 5: YOLOv8 Inference ===
# NOTE: This part uses FAKE image paths. Replace with real validation paths for a true evaluation.
try:
    yolo_disease = YOLO("outputs/crop_diseases_aug_model/weights/best.pt")
    yolo_insect = YOLO("outputs/crop_insects_aug_model/weights/best.pt")
except Exception as e:
    print(f"❌ Error loading YOLO model: {e}")
    exit()

def predict_yolo(model, image_path, task):
    # Create a dummy blank image if the path doesn't exist for demonstration
    if not os.path.exists(image_path):
        dummy_img = np.zeros((640, 640, 3), dtype=np.uint8)
        results = model.predict(dummy_img, save=False, verbose=False)
    else:
        results = model.predict(image_path, save=False, verbose=False)

    if task == "segment":
        return len(results[0].masks) if results[0].masks else 0
    else: # detect
        return len(results[0].boxes) if results[0].boxes else 0

# Extract image filenames from the validation set
val_disease_images = val_disease["Image"].values
val_insect_images = val_insect["Image"].values

print("\nRunning YOLO Inference (using dummy images as placeholders)...")
# Predict using YOLO (True if 1 or more detections, False otherwise)
yolo_disease_preds = np.array([
    predict_yolo(yolo_disease, os.path.join("Crop_Diseases_Aug/images/val", img), task="segment") > 0
    for img in val_disease_images
])
yolo_insect_preds = np.array([
    predict_yolo(yolo_insect, os.path.join("Crop_Insects_Aug/images/val", img), task="detect") > 0
    for img in val_insect_images
])
print("YOLO Inference complete.")

# === Step 6: Multimodal Fusion Logic (AND) ===
# This is the updated high-confidence logic.
# The `&` operator ensures both models must agree for a "Present" prediction.
final_disease_preds = yolo_disease_preds & (tabnet_disease_preds == 1)
final_insect_preds = yolo_insect_preds & (tabnet_insect_preds == 1)

# Convert boolean results to integers (1 for True, 0 for False)
final_disease_preds = final_disease_preds.astype(int)
final_insect_preds = final_insect_preds.astype(int)


# Ground Truths
true_disease = y_val_disease
true_insect = y_val_insect

# === Step 7: Evaluation ===
print("\n🌾 High-Confidence Multimodal Fusion Evaluation (AND Logic)")
print("---" * 15)

print("\n🦠 Disease Prediction:")
print(f"Accuracy: {accuracy_score(true_disease, final_disease_preds):.2%}")
print(classification_report(true_disease, final_disease_preds, target_names=["Not Present", "Present"]))

print("\n🐛 Insect Prediction:")
print(f"Accuracy: {accuracy_score(true_insect, final_insect_preds):.2%}")
print(classification_report(true_insect, final_insect_preds, target_names=["Not Present", "Present"]))

# === Step 8: Combined Multimodal Accuracy ===
combined_accuracy = (
    accuracy_score(true_disease, final_disease_preds) +
    accuracy_score(true_insect, final_insect_preds)
) / 2
print("\n" + "---" * 15)
print(f"✅ Combined Multimodal Accuracy: {combined_accuracy:.2%}")

✅ CSV datasets loaded successfully.


c:\Users\moham\Agrithon\venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Running YOLO Inference (using dummy images as placeholders)...
YOLO Inference complete.

🌾 High-Confidence Multimodal Fusion Evaluation (AND Logic)
---------------------------------------------

🦠 Disease Prediction:
Accuracy: 50.00%
              precision    recall  f1-score   support

 Not Present       0.50      1.00      0.67        15
     Present       0.00      0.00      0.00        15

    accuracy                           0.50        30
   macro avg       0.25      0.50      0.33        30
weighted avg       0.25      0.50      0.33        30


🐛 Insect Prediction:
Accuracy: 53.33%
              precision    recall  f1-score   support

 Not Present       0.53      1.00      0.70        16
     Present       0.00      0.00      0.00        14

    accuracy                           0.53        30
   macro avg       0.27      0.50      0.35        30
weighted avg       0.28      0.53      0.37        30


---------------------------------------------
✅ Combined Multimodal Acc

c:\Users\moham\Agrithon\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\moham\Agrithon\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\moham\Agrithon\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\moham\A